In [157]:
PYSPARK_PATH = '../spark/spark-2.4.3-bin-hadoop2.7/'

In [158]:
import os
import numpy as np

import findspark
findspark.init(PYSPARK_PATH)
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark.sql import *
from pyspark.sql.functions import col, lower, regexp_replace, split, size, UserDefinedFunction
from pyspark.sql.types import StringType, IntegerType
from functools import reduce
import re

In [159]:
# df_path = "csv_data/top-10-sample-output.csv"
df_path = "csv_data/sample-output-ores.csv"
df_out_path = "{}_features.csv".format(df_path[:-4])
df = spark.read.csv(df_path, inferSchema=True, header=True, multiLine=True, escape='"')

### EDA + Feature Engineering

In [160]:
"""
Columns filtering
    Useful: sha1 (as identifier),  timestamp, title, text
    Questionable: user, comment, ip, id (there are different articles with the same id), parentid, restrictions
    Not useful (no unique info): model, format, ns, contributor, revision, restrictions
""" 

print("All columns:", df.columns)
print("Unique values for..")
for column in ["format", "model", "ns", "contributor", "revision", "restrictions"]:
    print("\t", column, ":", df.select(column).distinct().rdd.map(lambda r: r[0]).collect())
    
ores_weights = {'Stub': 1, 'Start': 2, 'C': 3, 'B': 4, 'GA': 5, 'FA': 6}
ores_scores = list(ores_weights.keys())
useful_columns = ["sha1", "timestamp", "title", "text"] + ores_scores
print("Useful columns:", useful_columns)

All columns: ['_c0', 'Unnamed: 0', 'comment', 'contributor', 'format', 'id', 'ip', 'model', 'ns', 'parentid', 'restrictions', 'revision', 'sha1', 'text', 'timestamp', 'title', 'username', 'revid', 'B', 'C', 'FA', 'GA', 'Start', 'Stub']
Unique values for..
	 format : ['text/x-wiki']
	 model : ['wikitext']
	 ns : [12, 6, 4, 100, 118, 0]
	 contributor : [None, '  ', ' ']
	 revision : ['        ', '       ', '         ', '          ']
	 restrictions : ['edit=autoconfirmed:move=sysop', 'edit=autoconfirmed:move=autoconfirmed', 'edit=sysop:move=sysop', None, 'move=:edit=', 'sysop', 'move=sysop', 'move=sysop:edit=sysop']
Useful columns: ['sha1', 'timestamp', 'title', 'text', 'Stub', 'Start', 'C', 'B', 'GA', 'FA']


In [161]:
clean_df = df[useful_columns]
clean_df.printSchema()
print("Size of the DataFrame: {} records".format(clean_df.count()))

root
 |-- sha1: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- Stub: double (nullable = true)
 |-- Start: double (nullable = true)
 |-- C: double (nullable = true)
 |-- B: double (nullable = true)
 |-- GA: double (nullable = true)
 |-- FA: double (nullable = true)

Size of the DataFrame: 38842 records


In [162]:
df_features = clean_df

In [163]:
df_features.show(4)

+--------------------+-------------------+-----------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|                sha1|          timestamp|      title|                text|                Stub|               Start|                  C|                  B|                 GA|                 FA|
+--------------------+-------------------+-----------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|5ib24sew99ceyko33...|2019-04-11 16:19:39|The X-Files|{{About|the telev...|0.003073017103110...|0.007612710147196926|0.02019868704430509| 0.0510891495437035|0.31740231803291463| 0.6006241181287694|
|q2nngvea32gwowk9y...|2019-04-25 23:53:46|Third World|{{mergefrom|Third...|0.006592174737638895| 0.03438149756654297|  0.452702637198141|0.20820061897204512| 0.2271170545728272|0.07100601695280477|
|ihx6i3puo

In [164]:
def words_counts(df):
    return df.withColumn('n_words', size(split(col('text'), ' ')))

df_features = words_counts(df_features)

In [165]:
"""
Headings counting
Syntaxis:
    ==Level 2==
    ===Level 3===
    ====Level 4====
    =====Level 5=====
    ======Level 6======
"""

def single_head_level_count(text, level):
    assert level in range(2,7)
    pattern = "=" * level
    pattern = pattern + "[a-zA-Z0-9.,!? ]+" + pattern
    return size(split(text, pattern=pattern))-1

def count_headings(df):
    return reduce(
        lambda df, level: df.withColumn("level{}".format(level),
                                        single_head_level_count(col("text"), level)),
        range(2,7), df)
    
df_features = count_headings(df_features)

In [166]:
"""
Citation counting
Syntaxis:
    {{cite {book}(.*?)}}
    {{cite {journal}(.*?)}}
"""

def citation_counter(citation_source):
    def _count_citations(text):
        matches = re.findall(f"{{cite {citation_source}(.*?)}}", text, re.IGNORECASE)
        return len(matches)
    return _count_citations

book_citations_count = UserDefinedFunction(citation_counter("book"), IntegerType())
journal_citations_count = UserDefinedFunction(citation_counter("journal"), IntegerType())

df_features = df_features.withColumn("book_citations", book_citations_count("text"))\
  .withColumn("journal_citations", journal_citations_count("text"))

In [167]:
'''Internal Links:
    [[A]] -- internal reference to an article titled A
    [[A|B]] -- internal reference to an article titled A (written as B)
    [[A#C|B]] -- internal reference to a section C of an article titled A (written as B)'''

def count_internal_links(df):
    
    pattern = "\[\[[a-zA-Z0-9.,!? ]+\]\]"
    pattern += "|\[\[[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\]\]"
    pattern += "|\[\[[a-zA-Z0-9.,!? ]+#[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\]\]"
    
    return df.withColumn("n_internal_links",
                         size(split(col('text'), pattern=pattern))-1)

df_features = count_internal_links(df_features)

In [168]:
'''External Links:
    https://www.google.com -- simple link
    [https://www.google.com] -- link (reference)
    [https://www.google.com A] -- reference written as A
    <ref name="B">[https://www.google.com A]</ref> -- reference A written as B, can be referenced again like:
    <ref name="B" /> -- reference to the source B
    <ref>Lots of words</ref> -- reference without a link
    {{sfnm|1a1=Craig|1y=2005|1p=14|2a1=Sheehan|2y=2003|2p=85}} -- external reference
    Example:
        {{sfnm|1a1=McLaughlin|1y=2007|1p=59|2a1=Flint|2y=2009|2p=27}} -- McLaughlin 2007, p. 59; Flint 2009, p. 27.
        {{sfnm|1a1=Craig|1y=2005|1p=14|2a1=Sheehan|2y=2003|2p=85}} -- Craig 2005, p. 14; Sheehan 2003, p. 85.'''

def count_external_links(df):
    
    pattern = 'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    pattern += '|\[https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+\]'
    pattern += '|\[https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+\ [a-zA-Z0-9.,!? ]+]'
    pattern += '<ref name="[a-zA-Z0-9.,!? ]+">\[https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+\]'
    
    # template of the external reference
    #template = '\{\{sfnm\|1a1=[a-zA-Z]+\|1y=[0-9]+\|1p=[0-9]+\|2a1=[a-zA-Z]+\|2y=[0-9]+\|2p=[0-9]+\}\}'
    
    # <ref name="B" /> - this form use information from other reference, so we didn't count it again
    # <ref>Lots of words</ref> - reference without a link will be in the other feature
    
    return df.withColumn("n_external_links",
                         size(split(col('text'), pattern=pattern))-1)

df_features = count_external_links(df_features)

In [169]:
"""
Paragraphs
"""

def count_paragraphs(df):
    
    # filter the basic wikipedia syntaxis
    pattern_filtering = '\n\n\{\{.*\}\}\n\n|\n\n\[\[.*\]\]\n\n|\n\n={1,7}.*={1,7}\n\n'
    # split by two enters
    pattern_splitting = '\n\n'

    return df.withColumn('n_paragraphs', size(split(regexp_replace(col('text'), 
                                                                   pattern_filtering, ''), 
                                                    pattern_splitting))-1)

df_features = count_paragraphs(df_features)

In [170]:
'''<ref>Lots of words</ref> -- reference without a link
{{cn}} -- citation needed'''

def count_unreferenced(df):
    
    # citation needed and references without link
    pattern = '\{\{cn\}\}|<ref>[a-zA-Z0-9.,!? ]+</ref>'
    
    return df.withColumn('n_unreferenced', size(split(col('text'), pattern))-1)

df_features = count_unreferenced(df_features)

In [171]:
'''[[Category:Category name]]
[[:Category:Category name]]
[[:File:File name]]'''

def count_categories(df):
    
    #using template
    pattern = '\[\[:?Category:[a-zA-Z0-9.,\-!?\(\) ]+\]\]'
    
    return df.withColumn('n_categories', size(split(col('text'), pattern))-1)

df_features = count_categories(df_features)

In [172]:
'''
    [[File: | thumb  | upright | right | alt= | caption ]]
'''

def count_of_images(df):
    any_text = "[a-zA-Z0-9.,!? ]+ \] "
    pattern = "\[[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\|[a-zA-Z0-9.,!? ]+\]"
    return df.withColumn("n_images", size(split(col('text'), pattern=pattern))-1)

df_features = count_of_images(df_features)

In [173]:
df_features.printSchema()

root
 |-- sha1: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- Stub: double (nullable = true)
 |-- Start: double (nullable = true)
 |-- C: double (nullable = true)
 |-- B: double (nullable = true)
 |-- GA: double (nullable = true)
 |-- FA: double (nullable = true)
 |-- n_words: integer (nullable = false)
 |-- level2: integer (nullable = false)
 |-- level3: integer (nullable = false)
 |-- level4: integer (nullable = false)
 |-- level5: integer (nullable = false)
 |-- level6: integer (nullable = false)
 |-- book_citations: integer (nullable = true)
 |-- journal_citations: integer (nullable = true)
 |-- n_internal_links: integer (nullable = false)
 |-- n_external_links: integer (nullable = false)
 |-- n_paragraphs: integer (nullable = false)
 |-- n_unreferenced: integer (nullable = false)
 |-- n_categories: integer (nullable = false)
 |-- n_images: integer (nullable = false)



In [174]:
features_names = ['title',
                  'Stub',
                  'Start',
                  'C',
                  'B',
                  'GA',
                  'FA',
                  'n_words',
                  'level2',
                  'level3',
                  'level4',
                  'level5',
                  'level6',
                  'book_citations',
                  'journal_citations',
                  'n_internal_links',
                  'n_external_links',
                  'n_paragraphs',
                  'n_unreferenced',
                  'n_categories',
                  'n_images'
                 ]

df_features = df_features.select(list(map(lambda x: df_features[x].cast('double') if x != 'title' else df_features[x], 
                                          features_names)))

In [175]:
for feature in features_names:
    
    df_features = df_features.filter(df_features[feature].isNotNull())

In [176]:
df_features.toPandas().to_csv(df_out_path)

##  Clustering

In [177]:
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.feature import PCA
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

In [178]:
vectorAssembler = VectorAssembler(inputCols=features_names[1:],
                                  outputCol="features")

df_features = vectorAssembler.setHandleInvalid("skip").transform(df_features)

In [180]:
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(df_features)

# rescale each feature to range [min, max].
scaled_features = scalerModel.transform(df_features)

In [183]:
scaled_features = scaled_features.select('scaled_features')

In [185]:
scaled_features.limit(20).collect()

[Row(scaled_features=DenseVector([0.0018, 0.0054, 0.0183, 0.0559, 0.3611, 0.6502, 0.4907, 0.1323, 0.107, 0.0617, 0.0, 0.0, 0.0821, 0.0, 0.2171, 0.2139, 0.1974, 0.0, 0.1747, 0.5])),
 Row(scaled_features=DenseVector([0.0054, 0.0371, 0.5141, 0.2426, 0.258, 0.0761, 0.0528, 0.0389, 0.0, 0.0, 0.0, 0.0, 0.0321, 0.0062, 0.0291, 0.0045, 0.0494, 0.0051, 0.0262, 0.5])),
 Row(scaled_features=DenseVector([0.0022, 0.0065, 0.0421, 0.1143, 0.5523, 0.3917, 0.267, 0.0934, 0.0576, 0.0, 0.0, 0.0, 0.025, 0.0531, 0.1191, 0.0839, 0.2017, 0.0, 0.2314, 0.5])),
 Row(scaled_features=DenseVector([0.0037, 0.0105, 0.0694, 0.2049, 0.5469, 0.283, 0.1301, 0.07, 0.0329, 0.0123, 0.0, 0.0, 0.0429, 0.0938, 0.0628, 0.0385, 0.0858, 0.0, 0.0087, 0.5])),
 Row(scaled_features=DenseVector([0.0046, 0.0304, 0.3329, 0.4951, 0.0672, 0.2051, 0.0748, 0.0389, 0.0, 0.0, 0.0, 0.0, 0.0107, 0.0031, 0.0524, 0.0136, 0.0515, 0.0, 0.0044, 0.5])),
 Row(scaled_features=DenseVector([0.0032, 0.0295, 0.6133, 0.3768, 0.0959, 0.0228, 0.0441, 0.0467,

In [187]:
# Trains a bisecting k-means model.
bkm = BisectingKMeans(featuresCol='scaled_features').setK(3).setSeed(1)
model = bkm.fit(scaled_features)

# Evaluate clustering.
cost = model.computeCost(scaled_features)
print("Within Set Sum of Squared Errors = " + str(cost))

# Shows the result. 
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)

Py4JJavaError: An error occurred while calling o2703.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 195.0 failed 1 times, most recent failure: Lost task 0.0 in stage 195.0 (TID 7351, localhost, executor driver): java.lang.NullPointerException

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1889)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1877)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1876)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1876)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:926)
	at org.apache.spark.sc

In [ ]:
#get predictions and collect into the list
labels = model.transform(df_features).select('prediction')
labels = labels.collect() 
#create vanilla list with ints 
# instead of list with element type Rows
labels = list(map(lambda x: x.prediction, labels))

## PCA compressing

In [ ]:
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df_features)

compressed = model.transform(df_features).select("pcaFeatures")
compressed.show(10)

In [ ]:
compressed = compressed.toPandas()
compressed.pcaFeatures = compressed.pcaFeatures.apply(lambda x: np.array(x))
compressed = compressed.pcaFeatures.values
compressed = np.stack(compressed, axis=0)

In [ ]:
colors = {0: 'red', 1: 'green', 2: 'blue'}
colors = list(map(lambda x: colors[x], labels))